In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model        #used for loading model from the disk
import numpy as np
import cv2
import os

In [3]:
img_path=os.path.join(os.getcwd(),'./examples/70-with-mask.jpg')                            
face_cascade_path=os.path.join(os.getcwd(),'./face_detector/haarcascade_frontalface_default.xml')   #for face detection in the image
model_path=os.path.join(os.getcwd(),'./mask_detector.model')

In [4]:
model=load_model(model_path)

In [7]:
img=cv2.imread(img_path)
face_cascade=cv2.CascadeClassifier(face_cascade_path)
gry=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)               #conversion to grayScale is required as a preprocessing step for harrCascade
faces=face_cascade.detectMultiScale(gry,1.1,4)         #returns 4 coordinates of the face detected

for (x,y,w,h) in faces:

    face_roi=img[y:y+h,x:x+w]
    face_gry=cv2.cvtColor(face_roi,cv2.COLOR_BGR2RGB)
    
    face=cv2.resize(face_gry,(224,224))
    face=img_to_array(face)
    face=preprocess_input(face)
    face=np.expand_dims(face,axis=0)                   #this step is very very imp as the input tensor to the model is of the form (no. of examples,rows,cols,channels) in case of channel's first.
    #the np.expand_dims adds an extra dimension which indicates a single example of the image on which inference has to be done

    with_mask,without_mask=model.predict(face)[0]      #binary softmax output[prediction]===> [with_mask_class_probability,without_mask_class_probability]; 0 is used because it is for a single example
   
    label= 'mask' if with_mask>without_mask else 'without_mask'     # => get the max value then decide the class 
    color=  (0,255,0) if with_mask>without_mask else (0,0,255)  
    
    cv2.rectangle(img,(x,y),(x+w,y+h),color,3)
    y_end=y-5 if y-5>5 else y+5                       #this is done to deal with the window border problems 
    cv2.putText(img,label,(x+10,y_end),cv2.FONT_HERSHEY_PLAIN,2,color,2)
    
    
    
cv2.imshow('feed',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [8]:
#carrying out the detection using the webcam 
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('./face_detector/haarcascade_frontalface_default.xml')

while cap.isOpened():
    ret,frame=cap.read()
    gry=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gry,1.1,4)
    
    for (x,y,w,h) in faces:
        face_roi=frame[y:y+h,x:x+w]
        face_gry=cv2.cvtColor(face_roi,cv2.COLOR_BGR2RGB)
        face_gry=img_to_array(face_gry)
        face_gry=cv2.resize(face_gry,(224,224))
        face_gry=preprocess_input(face_gry)
        face_gry=np.expand_dims(face_gry,axis=0)
        
        with_mask,without_mask=model.predict(face_gry)[0]
        
        label='with mask' if with_mask>without_mask else 'without mask'
        color=(0,255,0) if with_mask>without_mask else (0,0,255)
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)
        y_end=y-5 if y-5>5 else y+5
        cv2.putText(frame,label,(x+8,y_end),cv2.FONT_HERSHEY_PLAIN,2,color,2)
        
        cv2.imshow('LIVE',frame)
        if(cv2.waitKey(1)==ord('q')):
            cap.release()
            cv2.destroyAllWindows()
            break;
